# **Tutorial used : https://www.youtube.com/watch?v=GFSiL6zEZF0**

In [ ]:

import pandas as pd
import pandas_datareader as web
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
crypto_currency = "BTC"
against_currency = "GBP"

# get the data from Yahoo finance for the crypto currency and a real world currency
start = dt.datetime(2016,1,1)
end = dt.datetime.now()
data = web.DataReader(f"{crypto_currency}-{against_currency}", 'yahoo', start, end)
data.head()

In [ ]:
# prepare the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_date = scaler.fit_transform(data["Close"].values.reshape(-1,1))

prediction_length = 60
x_train, y_train = [], []

# fill the training data
for i in range(prediction_length, len(scaled_date)):
    x_train.append(scaled_date[i-prediction_length:i, 0])
    y_train.append(scaled_date[i, 0])

# convert the data into numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
# reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [ ]:

# create neural network
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))


# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32, verbose=2)

In [ ]:
# testing the model
test_start = dt.datetime(2021,1,1)
test_end = dt.datetime.now()

test_data = web.DataReader(f"{crypto_currency}-{against_currency}", 'yahoo', test_start, test_end)
actual_price = test_data["Close"].values
total_dataset = pd.concat((data["Close"], test_data["Close"]), axis=0)

model_input = total_dataset[len(total_dataset) - len(test_data) - prediction_length:].values
model_input = model_input.reshape(-1, 1)
model_input = scaler.fit_transform(model_input)



In [ ]:
print(len(total_dataset),len(test_data), prediction_length)

In [ ]:
# make a prediction
x_test = []
for i in range(prediction_length, len(model_input)):
    x_test.append(model_input[i-prediction_length:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# Use the model to predict the future
prediction = model.predict(x_test)
prediction = scaler.inverse_transform(prediction)

In [ ]:
# visialize the results
plt.plot(actual_price[59:110], color='red', label='Actual Price')
plt.plot(prediction[59:110], color='blue', label='Predicted Price')

plt.title(f"{crypto_currency}-{against_currency}")
plt.xlabel("Time")
plt.ylabel("Price")
plt.legend(loc='best')

TESTING WITH DIFFERENT DATASET

In [ ]:
csv = pd.read_csv("../BTC-Historic-data.csv") # cleaning function to trim and change the date format of the data

In [ ]:
def trim_date(ts):
    date_as_string = str(dt.datetime.fromtimestamp(ts))  # cast to string
    return date_as_string.split(" ")[0]
    # return dt.datetime.strptime(trimmed_date, '%Y-%m-%d').strftime('%d/%m/%Y')

df = csv.copy()

In [ ]:
df.drop(["Volume_(BTC)"],axis=1, inplace=True)
df.rename(columns={'Timestamp': 'Date', 'Volume_(Currency)': 'Volume', 'Weighted_Price':'Adj Close'}, inplace=True)
df = df[csv['Close'].notna()]
df["Date"] = df["Date"].apply(trim_date)

df = df[df["Date"] == "2021-01-01"]

df = df[["Date","High", "Low", "Open", "Close", "Volume","Adj Close"]]

print(df.head(), df.columns)
print(test_data.head(), test_data.columns)